# Парсинг новостей

## Загрузка новостей с сайта vvsu

In [135]:
!pip install requests openpyxl bs4 pandas nltk numpy pymorphy2 tqdm ufal.udpipe

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for bs4: filename=bs4-0.0.1-py3-none-any.whl size=1264 sha256=4a150026e3e328752c0881a2267599ec95207e72ff16e85f39f1341612eb4963
  Stored in directory: c:\users\solad\appdata\local\pip\cache\wheels\e4\62\1d\d4d1bc4f33350ff84227f89b258edb552d604138e3739f5c83
Successfully built bs4


In [3]:
import requests
from bs4 import BeautifulSoup
from openpyxl import Workbook

In [4]:
# Создаем новый Excel-файл
workbook = Workbook()
sheet = workbook.active
sheet.append(['Название новости', 'Дата', 'Ссылка', 'Текст'])

base_url = 'https://vvsu.ru/latests/page/'
year = 2022
page_number = 1:
stop_condition = False

year_to_stop = year - 1

while not stop_condition:
    url = base_url + str(page_number)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    news_list = soup.find_all('div', class_='col-sm-12 col-xs-8')

    for news in news_list:
        news_title = news.find('a', class_='brand-link').text
        news_date = news.find('p', class_='date').text
        news_link = 'https://vvsu.ru' + news.find('a', class_='brand-link')['href']
        
        # Проверяем, что новость раньше 2022 года
        if year_to_stop in news_date:
            stop_condition = True
            break
        
        # Парсим страницу новости для получения текста
        news_response = requests.get(news_link)
        news_soup = BeautifulSoup(news_response.text, 'html.parser')
        news_content = news_soup.find('article', class_='col-md-12 m-b-5 p-r-5 bottom-content')
        news_text = news_content.get_text(strip=True)
        
        # Заносим новость и текст в Excel-файл
        sheet.append([news_title, news_date, news_link, news_text])

    page_number += 1
    
# Сохраняем в Excel-файл
workbook.save('news_with_text.xlsx')

## Поиск по vlnews

In [6]:
import urllib.parse
    

"""Функция для парсинга по ключевому слову и до определенного года включительно, на сайте vlnews.
А также выгрузка файла excel с названием новости, ссылкой, датой и самим текстом новости."""

def search_on_vlnews_on_word_to_year(word, year):
    
    workbook_media = Workbook()
    sheet_media = workbook_media.active
    sheet_media.append(['Название новости', 'Дата', 'Ссылка', 'Текст'])

    encoded_word = urllib.parse.quote(word, encoding='utf-8')

    year_to_stop = year - 1

    base_url_media = f'https://www.newsvl.ru/find/?q={encoded_word}&page='
    page_number_media = 1
    news_number = 1
    stop_condition = False

    while not stop_condition:
        url = base_url_media + str(page_number_media)
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
    
    
        news_list = soup.find_all('div', class_='story-list__item')
    
        if not news_list:
            stop_condition = True
            break
    
        for news in news_list:
            news_info = news.find('div', class_='story-list__item-info')
    
            news_date = news_info.find('span', class_='story-list__item-date').text
    
            news_content = news.find ('div', class_='story-list__item-content')
            news_title_with_link =  news_content.find('h3', class_ = 'story-list__item-title')
    
            news_title =  news_title_with_link.find('a').text 
            news_link = 'https://www.newsvl.ru' + news_title_with_link.find('a')['href']
            
        
            if str(year_to_stop) in news_date:
                stop_condition = True
                break
        
            news_response = requests.get(news_link)
            news_soup = BeautifulSoup(news_response.text, 'html.parser')
            news_content_text = news_soup.find('div', class_ = 'story__text')
            news_text = news_content_text.get_text(strip = True)
            
            print('{0} новость, ссылка: {1}'.format(news_number, news_link))
            
            news_number += 1
    
            # Заносим новость и текст в Excel-файл
            sheet_media.append([news_title, news_date, news_link, news_text])
        
        page_number_media += 1
        
     #Сохраняем Excel-файл   
    workbook_media.save(f'news_vlnews_on_{word}.xlsx')

In [28]:
search_on_vlnews_on_word_to_year('ВГУЭС', 2022)

In [29]:
search_on_vlnews_on_word_to_year('ВВГУ', 2022)

## Обьеденение таблиц по media и загрузка таблицы с новостями из vvsu

In [1]:
import pandas as pd

In [2]:
df1 = pd.read_excel('news_vlnews_on_ВВГУ.xlsx')
df2 = pd.read_excel('news_vlnews_on_ВГУЭС.xlsx')

merged_df_media = pd.concat([df1, df2], ignore_index=True)

In [3]:
merged_df_media

,Название новости,Дата,Ссылка,Текст
0,Первую в истории университета киберспортивную ...,"20:38, 23 июня 2023",https://www.newsvl.ru/vlad/2023/06/23/217803/,Во Владивостокском государственном университет...
1,"Несмотря на жару и очереди, первый VK Fest с у...","16:30, 17 июня 2023",https://www.newsvl.ru/vlad/2023/06/17/217680/,"В субботу, 17 июня, на набережной Цесаревича в..."
2,Правую полосу на улице Гоголя огородили бетонн...,"11:43, 5 июня 2023",https://www.newsvl.ru/vlad/2023/06/05/217443/,Дорожные рабочие огородили бетонными блоками к...
3,Передел рыбопромышленной отрасли или «парадокс...,"20:02, 25 мая 2023",https://www.newsvl.ru/vlad/2023/05/25/217242/,«Парадоксы государственного управления» в рыбн...
4,«Звезда» городского субботника: во Владивосток...,"14:55, 15 апреля 2023",https://www.newsvl.ru/vlad/2023/04/15/216463/,15 апреля во Владивостоке провели масштабные с...
...,...,...,...,...
94,"Мэрия Владивостока показала, как может выгляде...","11:26, 7 февраля 2022",https://www.newsvl.ru/vlad/2022/02/07/206407/,Администрация Владивостока опубликовала визуал...
95,Реновация во Владивостоке начнётся сносом част...,"16:22, 26 января 2022",https://www.newsvl.ru/vlad/2022/01/26/206055/,Пилотный для Владивостока проект реновации – з...
96,Ford пробил дорожное ограждение и врезался в т...,"15:57, 25 января 2022",https://www.newsvl.ru/accidents/2022/01/25/206...,"Сегодня, 25 января, водитель автомобиля Ford K..."
97,На Гоголя загорелось здание рядом с автокафе C...,"22:44, 24 января 2022",https://www.newsvl.ru/vlad/2022/01/24/205998/,"В понедельник, 24 января, во Владивостоке, в р..."


In [4]:
#Проверяем на наличие дубликатов
merged_df_media.duplicated().sum()

11

In [5]:
#Убираем дубликаты новостей
merged_df_media.drop_duplicates(inplace=True)

In [6]:
df_vvsu = pd.read_excel('news_vvsu_with_text.xlsx')
df_vvsu

,Название новости,Дата,Ссылка,Текст
0,Всероссийская ярмарка трудоустройства-2023 в ВВГУ,23 июня 2023,https://vvsu.ru/latest/article/2151056183/Vser...,Всероссийская ярмарка вакансий собрала в стена...
1,"Высокобалльники, поступающие в ВВГУ, получат и...",23 июня 2023,https://vvsu.ru/latest/article/2151055493/Vyso...,Руководство Владивостокского государственного ...
2,ВВГУ продолжает расширять связи и сотрудничест...,23 июня 2023,https://vvsu.ru/latest/article/2151053935/VVGU...,ВВГУ посетила делегация из Внутренней Монголии...
3,"Студенты ВВГУ проверили, чем мы дышим",22 июня 2023,https://vvsu.ru/latest/article/2151048613/Stud...,Студенты ВВГУ изучили влияние автомобильного п...
4,Ректор ВВГУ Татьяна Терентьева: Наши выпускник...,22 июня 2023,https://vvsu.ru/latest/article/2151047921/Rekt...,В ВВГУ – горячая пора предзащиты выпускных ква...
...,...,...,...,...
594,Ректор ВГУЭС Татьяна Терентьева в федеральном ...,12 января 2022,https://vvsu.ru/latest/article/2149459929/Rekt...,Как университет выстраивает взаимодействие меж...
595,"«Я вижу каменное небо»: премьера фильма, посвя...",12 января 2022,https://vvsu.ru/latest/article/2149457798/Ja_v...,14 января 2022 года исполняется 131 год со дня...
596,Вдохновляющая история успеха: выпускник ВГУЭС ...,10 января 2022,https://vvsu.ru/latest/article/2149440949/Vdok...,"Выпускник ВГУЭС 1995 года, владелец интернет-м..."
597,Ректор ВГУЭС Татьяна Терентьева отмечена благо...,10 января 2022,https://vvsu.ru/latest/article/2149454177/Rekt...,Правительство Приморского края в лице краевого...


# Обработка текста/Приведение к нормальной форме

In [43]:
import re
import nltk
import pymorphy2
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\solad\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\solad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\solad\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [44]:
text_media = merged_df_media['Текст']
text_vvsu = df_vvsu['Текст']
norm_text_media = []
norm_text_vvsu = []

In [45]:
# Приведение к нормальной форме текста из новостей медиа
morph=pymorphy2.MorphAnalyzer()
for s in tqdm(text_media):
    s1 = re.sub(r'[^\w\s]+|[\d]+', r'',s).strip()
    s1 = s1.lower()
    s1 = word_tokenize(s1)
    words=[]
    for i in s1:
        pv = morph.parse(i)
        words.append(pv[0].normal_form)
    sentence=' '.join(words)
    norm_text_media.append(sentence)

100%|██████████| 88/88 [00:21<00:00,  4.05it/s]


In [46]:
# Приведение к нормальной форме текста из новостей сайта ВВГУ
for s in tqdm(text_vvsu):
    s1 = re.sub(r'[^\w\s]+|[\d]+', r'',s).strip()
    s1 = s1.lower()
    s1 = word_tokenize(s1)
    words=[]
    for i in s1:
        pv = morph.parse(i)
        words.append(pv[0].normal_form)
    sentence=' '.join(words)
    norm_text_vvsu.append(sentence)

100%|██████████| 599/599 [01:34<00:00,  6.36it/s]


In [47]:
norm_text_media[0]

'в владивостокский государственный университет перерезание светодиодный лента торжественно открыть первый в история вуз кибер арена ввг площадка стать тренировочный база для университетский сборная по киберспорт уже осень здесь провести отборочный соревнование и определить состав команда для состязание различный уровнейв пятница июнь на шестой этаж главное корпус владивостокский государственный университет состояться торжественный открытие киберспортивный арена пространство где команда соревноваться между себя в компьютерный игра такой новшество в университет появиться после тот как сам учиться заявить о необходимость он введение создать киберарен помочь спонсор в сумма собрать около миллион рубль на реализация проектая не знать что такой киберарен я не понимать что такой киберспорт но когда молодёжь прийти и сказать что они это надо и это сегодняшний а тем более завтрашний день я понять что мы это надо сегодня я впервые вместе с вы заглянуть туда посмотреть что мы сделать но если это 

In [123]:
norm_text_vvsu[0]

'всероссийский ярмарка вакансия собрать в стена ввг работодатель соискатель и специалист по подбор персонал студент выпускник и всё кто озадачить поиск работа смочь пообщаться с представитель предприятие приморский край и повысить свой компетенция в область поиск работа на мастерклассахярмарка в концертный зал андерграунд торжественно открыть губернатор приморьяолег кожемяко приморский край располагать к жизнь и работа это не окраина как считать многие есть больший карьерный перспектива в судостроение в itсфера в экономика в сельский хозяйство в рыбопромысловый и торговый флот не надо бояться никакой работа в тот число и рабочий профессия с они вы всегда найти место в жизнь я сам начинать слесарь разряд если вы пройти всё ступень карьерный лестница из вы получиться хороший руководитель в наш время быть безработный стыдно особенно учитывать то что у мы в приморский край тысяча вакансийс приветственный слово обратиться к участник и гость ярмарка ректор ввг доктор экономический наука проф

In [124]:
stop_words = set(stopwords.words('russian'))

# Удаление стоп-слов из списка токенизированного текста новостей в norm_text_media
norm_text_media = [word_tokenize(news) for news in norm_text_media]
norm_text_media = [[word for word in news if word not in stop_words] for news in norm_text_media]

norm_text_vvsu = [word_tokenize(news) for news in norm_text_vvsu]
norm_text_vvsu = [[word for word in news if word not in stop_words] for news in norm_text_vvsu]

In [125]:
norm_text_media

[['владивостокский',
  'государственный',
  'университет',
  'перерезание',
  'светодиодный',
  'лента',
  'торжественно',
  'открыть',
  'первый',
  'история',
  'вуз',
  'кибер',
  'арена',
  'ввг',
  'площадка',
  'стать',
  'тренировочный',
  'база',
  'университетский',
  'сборная',
  'киберспорт',
  'осень',
  'провести',
  'отборочный',
  'соревнование',
  'определить',
  'состав',
  'команда',
  'состязание',
  'различный',
  'уровнейв',
  'пятница',
  'июнь',
  'шестой',
  'этаж',
  'главное',
  'корпус',
  'владивостокский',
  'государственный',
  'университет',
  'состояться',
  'торжественный',
  'открытие',
  'киберспортивный',
  'арена',
  'пространство',
  'команда',
  'соревноваться',
  'компьютерный',
  'игра',
  'новшество',
  'университет',
  'появиться',
  'учиться',
  'заявить',
  'необходимость',
  'введение',
  'создать',
  'киберарен',
  'помочь',
  'спонсор',
  'сумма',
  'собрать',
  'около',
  'миллион',
  'рубль',
  'реализация',
  'проектая',
  'знать',
  '

In [126]:
# Объединение всех слов из всех новостей в один список для norm_text_media
all_words_media = [word for news in norm_text_media for word in news]

# Объединение всех слов из всех новостей в один список для norm_text_vvsu
all_words_vvsu = [word for news in norm_text_vvsu for word in news]

# Анализ популярных слов/словосочетаний

### Популярные слова

In [127]:
from collections import Counter

# Подсчет частоты встречаемости словосочетаний в списке всех слов для norm_text_media
word_counts_media = Counter(all_words_media)

# Получение наиболее популярных словосочетаний для norm_text_media
popular_phrases_media = word_counts_media.most_common(10)
word_counts_vvsu = Counter(all_words_vvsu)
popular_phrases_vvsu = word_counts_vvsu.most_common(10)

# Вывод результатов для norm_text_media
print("Популярные слова для norm_text_media:")
for phrase, count in popular_phrases_media:
    print(f'{phrase}: {count} раз')

Популярные слова для norm_text_media:
год: 397 раз
это: 355 раз
владивосток: 271 раз
который: 270 раз
свой: 158 раз
также: 135 раз
приморский: 133 раз
новый: 129 раз
человек: 128 раз
весь: 120 раз


In [129]:
# Вывод результатов для norm_text_vvsu
print("Популярные слова для norm_text_vvsu:")
for phrase, count in popular_phrases_vvsu:
    print(f'{phrase}: {count} раз')

Популярные слова для norm_text_vvsu:
студент: 1511 раз
это: 1382 раз
университет: 1306 раз
который: 1264 раз
год: 1159 раз
ввг: 1093 раз
свой: 966 раз
работа: 936 раз
проект: 905 раз
вгуэс: 840 раз


### Популярные словосочетания

In [130]:
from nltk import ngrams

# Функция для создания биграмм из списка слов
def create_bigrams(words):
    return list(ngrams(words, 2))

In [131]:
# Создание биграмм для norm_text_media
all_bigrams_media = create_bigrams(all_words_media)

# Подсчет частоты встречаемости биграмм
bigram_counts_media = Counter(all_bigrams_media)

# Получение наиболее популярных словосочетаний для norm_text_media
popular_phrases_media = bigram_counts_media.most_common(10)
print("Популярные словосочетания для сайта vlnews:")
for phrase, count in popular_phrases_media:
    print(f'{phrase}: {count} раз')

Популярные словосочетания для сайта vlnews:
('приморский', 'край'): 44 раз
('администрация', 'владивосток'): 38 раз
('светланский', 'спектакль'): 32 раз
('мариинский', 'театр'): 30 раз
('это', 'год'): 28 раз
('летие', 'владивосток'): 26 раз
('приморский', 'краевой'): 26 раз
('билет', 'приморский'): 26 раз
('млн', 'рубль'): 24 раз
('коммунист', 'россия'): 24 раз


In [132]:
all_bigrams_vvsu = create_bigrams(all_words_vvsu)
bigram_counts_vvsu = Counter(all_bigrams_vvsu)
popular_phrases_vvsu = bigram_counts_vvsu.most_common(10)
print("Популярные словосочетания для сайта vvsu:")
for phrase, count in popular_phrases_vvsu:
    print(f'{phrase}: {count} раз')

Популярные словосочетания для сайта vvsu:
('экономический', 'наука'): 225 раз
('приморский', 'край'): 222 раз
('дальний', 'восток'): 212 раз
('доктор', 'экономический'): 192 раз
('наш', 'университет'): 190 раз
('это', 'год'): 172 раз
('государственный', 'университет'): 152 раз
('высокий', 'образование'): 113 раз
('международный', 'бизнес'): 113 раз
('принять', 'участие'): 111 раз


In [136]:
# Создание списка кортежей для popular_phrases_media и vvsu
popular_phrases_media_list = [(phrase[0][0], phrase[0][1], phrase[1]) for phrase in popular_phrases_media]

popular_phrases_vvsu_list = [(phrase[0][0], phrase[0][1], phrase[1]) for phrase in popular_phrases_vvsu]

In [137]:
# Создание DataFrame для popular_phrases_media
df_media = pd.DataFrame(popular_phrases_media_list, columns=['Phrase1', 'Phrase2', 'Count'])
print("Популярные словосочетания для сайта vlnews:")
df_media

Популярные словосочетания для сайта vlnews:


,Phrase1,Phrase2,Count
0,приморский,край,44
1,администрация,владивосток,38
2,светланский,спектакль,32
3,мариинский,театр,30
4,это,год,28
5,летие,владивосток,26
6,приморский,краевой,26
7,билет,приморский,26
8,млн,рубль,24
9,коммунист,россия,24


In [138]:
df_vvsu = pd.DataFrame(popular_phrases_vvsu_list, columns=['Phrase1', 'Phrase2', 'Count'])
print("Популярные словосочетания для сайта vvsu:")
df_vvsu

Популярные словосочетания для сайта vvsu:


,Phrase1,Phrase2,Count
0,экономический,наука,225
1,приморский,край,222
2,дальний,восток,212
3,доктор,экономический,192
4,наш,университет,190
5,это,год,172
6,государственный,университет,152
7,высокий,образование,113
8,международный,бизнес,113
9,принять,участие,111


## Выполнение анализа текста и нахождение сочетания глагол-существительное в предложениях.

In [7]:
import ufal.udpipe
from ufal.udpipe import Model, Pipeline, ProcessingError
from tqdm.auto import tqdm
from collections import defaultdict

In [164]:
def find_child_nouns(sentence, parent_index):
    """
    Находит дочерние существительные для указанного глагола в предложении.
    Аргументы:
        - sentence: предложение в виде объекта Sentence из библиотеки ufal.udpipe
        - parent_index: индекс слова, представляющего глагол
    Возвращает:
        - список индексов дочерних существительных
    """
    children = []
    for i, word in enumerate(sentence.words):
        if word.upostag == "NOUN" and any(case in word.feats for case in ["Acc", "Loc", "Dat"]):
            if i != parent_index:
                children.append(i)
    return children

def find_verb_noun_combinations(sentence, combinations):
    """
    Находит сочетания глагол-существительное в предложении и обновляет словарь combinations.
    Аргументы:
        - sentence: предложение в виде объекта Sentence из библиотеки ufal.udpipe
        - combinations: словарь для хранения сочетаний глагол-существительное
    """
    for i, word in enumerate(sentence.words):
        if word.upostag == "VERB":
            children = find_child_nouns(sentence, i)
            for child in children:
                combinations[word.lemma][sentence.words[child].form] += 1
                
def find_child_adjectives(sentence, parent_index):
    """
    Находит дочерние прилагательные для указанного существительного в предложении.
    Аргументы:
        - sentence: предложение в виде объекта Sentence из библиотеки ufal.udpipe
        - parent_index: индекс слова, представляющего существительное
    Возвращает:
        - список индексов дочерних прилагательных
    """
    children = []
    for i, word in enumerate(sentence.words):
        if word.head == parent_index and word.upostag == "ADJ":
            children.append(i)
    return children


def find_noun_adjective_combinations(sentence, combinations):
    """
    Находит сочетания существительное-прилагательное в предложении и обновляет словарь combinations.
    Аргументы:
        - sentence: предложение в виде объекта Sentence из библиотеки ufal.udpipe
        - combinations: словарь для хранения сочетаний существительное-прилагательное
    """
    for i, word in enumerate(sentence.words):
        if word.upostag == "NOUN":
            children = find_child_adjectives(sentence, i)
            for child in children:
                combinations[word.form][sentence.words[child].form] += 1


def process_texts(texts, model):
    """
    Обрабатывает список текстов и находит сочетания существительное-прилагательное и существительное-глагол.
    Аргументы:
        - texts: список текстов для обработки
        - model: объект модели из библиотеки ufal.udpipe
    Возвращает:
        - словарь сочетаний существительное-прилагательное и существительное-глагол
    """
    tokenizer = model.newTokenizer(model.DEFAULT)
    noun_adjective_combinations = defaultdict(lambda: defaultdict(int))
    verb_noun_combinations = defaultdict(lambda: defaultdict(int))
    for text in tqdm(texts):
        tokenizer.setText(text)
        while tokenizer.nextSentence(sentence, error):
            model.tag(sentence, model.DEFAULT)
            model.parse(sentence, model.DEFAULT)
            find_noun_adjective_combinations(sentence, noun_adjective_combinations)
            find_verb_noun_combinations(sentence, verb_noun_combinations)
    return noun_adjective_combinations, verb_noun_combinations

In [160]:
# Загрузка модели
model = Model.load("russian-syntagrus-ud-2.5-191206.udpipe")
sentence = ufal.udpipe.Sentence()
error = ufal.udpipe.ProcessingError()

In [165]:
vvsu_texts = df_vvsu['Текст']
vvsu_noun_adjective_combinations, vvsu_verb_noun_combinations = process_texts(vvsu_texts, model)

  0%|          | 0/599 [00:00<?, ?it/s]

In [175]:
# Получение словаря связок существительное-прилагательное
vvsu_noun_adjective = pd.DataFrame(
    [(parent, child, freq) for parent, children in vvsu_noun_adjective_combinations.items()
     for child, freq in children.items() if int(freq) > 30],
    columns=['parent', 'children', 'freq']
)
vvsu_noun_adjective.sort_values(by='freq', ascending=False, inplace=True)

# Получение словаря связок существительное-глагол
vvsu_verb_noun = pd.DataFrame(
    [(parent, child, freq) for parent, children in vvsu_verb_noun_combinations.items()
     for child, freq in children.items() if int(freq) > 20],
    columns=['parent', 'children', 'freq']
)
vvsu_verb_noun.sort_values(by='freq', ascending=False, inplace=True)

vvsu_verb_noun

,parent,children,freq
2,принять,участие,119
8,принимать,участие,59
6,занять,место,55
5,стать,году,39
7,давать,возможность,38
4,получить,знания,33
1,дать,возможность,32
9,благодарить,помощь,30
10,благодарить,подготовке,30
3,получить,возможность,29


In [177]:
vvsu_noun_adjective

,parent,children,freq
0,наук,экономических,225
6,бизнеса,международного,109
3,образования,высшего,97
2,университета,государственного,92
8,университет,государственный,73
9,технологий,информационных,72
1,университета,Владивостокского,51
4,образования,профессионального,49
17,политики,молодежной,47
5,языка,русского,44


In [183]:
# Обработка текстов из media
media_texts = merged_df_media['Текст']
media_noun_adjective_combinations, media_verb_noun_combinations = process_texts(media_texts, model)

  0%|          | 0/88 [00:00<?, ?it/s]

In [195]:
# Получение словаря связок существительное-прилагательное
media_noun_adjective = pd.DataFrame(
    [(parent, child, freq) for parent, children in media_noun_adjective_combinations.items()
     for child, freq in children.items() if int(freq) > 10],
    columns=['parent', 'children', 'freq']
)
media_noun_adjective.sort_values(by='freq', ascending=False, inplace=True)

# Получение словаря связок существительное-глагол
media_verb_noun = pd.DataFrame(
    [(parent, child, freq) for parent, children in media_verb_noun_combinations.items()
     for child, freq in children.items() if int(freq) > 10],
    columns=['parent', 'children', 'freq']
)
media_verb_noun.sort_values(by='freq', ascending=False, inplace=True)

media_noun_adjective

,parent,children,freq
4,театра,Мариинского,24
0,году,прошлом,19
3,сцена,Приморская,19
1,движения,дорожного,18
5,филармония,краевая,18
6,филармония,Светланская,18
9,м,кв.,18
11,людей»,«Новых,17
13,справедливость,социальную,14
10,состоянии,хорошем,13


In [196]:
media_verb_noun

,parent,children,freq
2,продажакотить,спектакль,18
1,мочь,году,16
0,принять,участие,14
